In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
import time
import timeit
from IPython.display import clear_output
from goodreads import client
import random
#gc = client.GoodreadsClient('insert key')

In [2]:
#Imports data, inserts columns, sets column dtype
df_all = pd.read_csv('books1.csv')
genre = pd.read_csv('genreslist.csv')

#Slice data - do not change function 
def slice_data(start_index,end_index): 
    df = df_all[start_index:end_index].reset_index(drop=True)
    return df

###Change below - change start and end index
start_index = 2
end_index = 5
df = slice_data(start_index,end_index)

In [3]:
#Lists 
bgenrelist = genre['genre'].tolist()
bookgenres = {}
broken = []

def book_description(df): 
    start = timeit.default_timer()
    for x in range(len(df)): 
        clear_output(wait=True)

        #Set initial values to None 
        description = None
        genrestring = None

        #Obtain isbn and book description
        try:    
            myisbn = int(df.loc[x][1])
            book = gc.book(isbn = myisbn)
            description = book.description
            df.loc[x,'description'] = description
        
        #Pulls list of shelves and removes shelves that are not on predefined genre list
            books = book._book_dict['popular_shelves']['shelf']
            index = range(len(books))
            genres = pd.DataFrame(books, columns=['isbn13','@name', '@count'], index=index)
            bookgenres[str(myisbn)]=genres
        
            genrelist = bookgenres[str(myisbn)]['@name'].tolist()
            genrelistkeep = []

            for i in genrelist:
                if i in bgenrelist:
                    genrelistkeep.append(i)
                else:
                    continue
                
            genrestring = str(genrelistkeep).strip('[]')
            if genrestring == "":
                genrestring = None
            df.loc[x,'genres'] = genrestring
        
        #If API fails to pull shelves or description, appends ISBN to list for further investigation
        except:
            broken.append(myisbn)          
            pass
        
        stop =  timeit.default_timer()  
        
        #Random rest time after ever 70 books
        if x % 70 == 0: 
            time.sleep(random.randint(3,5))
        
        #Times
        if x < 10: 
                expected_time = "Calculating..." 
        else: 
            stop1 = timeit.default_timer()
            expected_time = np.round( ( (stop1-start)/(x/len(df)) )/60, 2)

        print('Current progress:', np.round((x+1)/len(df)*100, 2),"%")
        print('Current progress: {} out of {} books.'.format(x+1,len(df)))
        print('Total run time:', np.round((stop-start)/60, 2), "minutes")
        print('Expected run time:', expected_time, "minutes")


In [4]:
#Scrape
book_description(df)

Current progress: 100.0 %
Current progress: 3 out of 3 books.
Total run time: 0.13 minutes
Expected run time: Calculating... minutes


In [5]:
#Save to csv
file_name = 'output_' + str(start_index) + '_' + str(end_index) + '.csv'
df.to_csv(file_name, index=False) 